<h1><center>Developing Python Backends for Machine Learning Applications</center></h1>
<h2><center>by Jaidev Deshpande</center></h2>
<h3><center>Data Scientist @ Cube26 Software Pvt Ltd</center></h3>

<div style="text-align: center">
<div id="social">
<div id="social_twitter">
    <a href="http://twitter.com/jaidevd"><img src="images/twitter-128.png" width="32" height="32">
    </a>
</div>
<div id="social_medium">
    <a href="http://medium.com/@jaidevd"><img src="images/medium.ico" width="32" height="32"></a>
</div>
<div id="social_github">
    <a href="http://github.com/jaidevd"><img src="images/mark-github-128.png" width="32" height="32"></a>
</div>
</div>
</div>

<div style="text-align: center"><font size="4"><strong>@jaidevd</strong></font></div>

# Example: How we started building ReosMessage

<ul>
    <li><h3>Classification of SMS into personal, transactional and spam</h3></li>
    <li><h3>Get a dump of a table from postgres</h3></li>
    <li><h3>Use simple Python scripts and some pandas magic to clean the data</h3></li>
    <li><h3>Use regular expressions to label the data</h3></li>
    <li><h3>Train sklearn estimators on the labeled data</h3></li>
    <li><h3>Crowdsource the evaluation of the predictions</h3></li>
    <li><h3>Dump the model coefficients to JSON</h3></li>
    <li><h3>Hand over the JSON to Android developers</h3></li>
</ul>

# Typical Data Processing Pipeline
![](images/flowchart.png)